In [ ]:
import os
import sys
import psutil
import traci
import time

# Ensure SUMO_HOME is set
if "SUMO_HOME" in os.environ:
    sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

# Function to kill old SUMO processes before starting
def kill_sumo():
    for process in psutil.process_iter(attrs=["pid", "name"]):
        if "sumo" in process.info["name"].lower():
            print(f"Killing SUMO process {process.info['name']} (PID: {process.info['pid']})")
            os.kill(process.info["pid"], 9)

kill_sumo()

# Run simulation
sumoBinary = "sumo/sumo/bin/sumo-gui"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

try:
    # Increase retries to prevent connection issues
    traci.start(sumoCmd, numRetries=20)

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        for veh_id in traci.vehicle.getIDList():
            try:
                # Access the battery level directly using traci.vehicle.getParameter
                battery_level = traci.vehicle.getParameter(veh_id, "device.battery.actualBatteryCapacity")
                max_battery = traci.vehicle.getParameter(veh_id, "device.battery.maximumBatteryCapacity")

                print(f"Vehicle {veh_id} Battery Level: {battery_level} kWh / " + {max_battery} + " kWh")
            except traci.TraCIException as e:
                print(f"Error retrieving battery level for {veh_id}: {e}")
                
except traci.exceptions.FatalTraCIError as e:
    print(f"TraCI connection lost: {e}")

except BrokenPipeError:
    print("BrokenPipeError: SUMO crashed or closed unexpectedly.")

finally:
    time.sleep(1)
    try:
        if traci.connection._connections:
            print("Closing TraCI connection properly...")
            traci.close()
    except Exception as e:
        print(f"TraCI was already closed: {e}")


 Retrying in 1 seconds
Vehicle veh0 Battery Level: 3000.00 kWh
Vehicle veh0 Battery Level: 2999.42 kWh
Vehicle veh0 Battery Level: 2998.83 kWh
Vehicle veh0 Battery Level: 3001.82 kWh
Vehicle veh0 Battery Level: 3005.15 kWh
Vehicle veh0 Battery Level: 3008.05 kWh
Vehicle veh0 Battery Level: 3010.51 kWh
Vehicle veh0 Battery Level: 3012.52 kWh
Vehicle veh0 Battery Level: 3014.09 kWh
Vehicle veh0 Battery Level: 3011.99 kWh
Vehicle veh0 Battery Level: 3009.25 kWh
Vehicle veh0 Battery Level: 3005.96 kWh
Vehicle veh1 Battery Level: 3000.00 kWh
Vehicle veh0 Battery Level: 3002.07 kWh
Vehicle veh1 Battery Level: 2999.66 kWh
Vehicle veh0 Battery Level: 2997.44 kWh
Vehicle veh1 Battery Level: 2998.73 kWh
Vehicle veh0 Battery Level: 2993.16 kWh
Vehicle veh1 Battery Level: 2997.22 kWh
Vehicle veh0 Battery Level: 2992.57 kWh
Vehicle veh1 Battery Level: 2995.12 kWh
Vehicle veh0 Battery Level: 2991.98 kWh
Vehicle veh1 Battery Level: 2992.43 kWh
Vehicle veh0 Battery Level: 2991.40 kWh
Vehicle veh1 Batt